In [ ]:
#Cleaning the environment
rm(list=ls())

#Set the working directory
library(rstudioapi)
#Get the path of current open file
current_path <- getActiveDocumentContext()$path
#Set the workong directory to the rilevant one
setwd(dirname(current_path))
print(getwd())

#Upload library
library(tidyverse)
library(ggplot2)
library(dplyr)
library(ggpubr)
library(caret)
library(readr)

library(stats)
library(cluster)
library(NbClust)
library(pvclust)

In [ ]:
#Upload the Dataset
customers <- read_csv("Mall_Customers.csv")
View(customers)


## DATA PRE-PROCESSING
table(is.na(customers)) #NO missing value
customers=customers[,-1] #delete the 'customer_id' variables

#I turn the Gender variable into a factor and assign: 0=Female, 1=Male
customers$Gender=as.factor(customers$Gender)
customers$Gender=factor(customers$Gender,levels = c('Female','Male'),labels = c(0,1))

#Rename the variables
colnames(customers)[3] <- "AnnualIncome"
colnames(customers)[4] <- "SpendingScore"
view(customers)


In [ ]:
##PRELIMINARY EXPLORATORY ANALYSIS

summary(customers)
str(customers) #check the structure of dataset;
dim(customers) #200 records and 5 variables;
colnames(customers)#VARIABLES: CUSTOMER_ID, GENDER. AGE, ANNUAL INCOME, SPENDING-SCORE;
head(customers)

#Plot the relation between male and female customers
dev.new()
ggplot(customers)+
  aes( x = Gender) + 
  geom_bar(fill = "lightblue") + 
  geom_hline(yintercept = 0, size=1, colour = "#444444") + 
  labs(title = "distribution of gender customers")
#add percentages
bb<-customers$Gender%>%table()%>%
  barplot(main="Gender Customers", ylab="Frequency",
          col=c("lightgreen",'lightblue'))
tab<-customers$Gender%>%table()
precentages<-tab%>%prop.table()%>%round(3)*100
txt<-paste0( precentages,'%')
text(bb,tab/2, labels=txt, cex=1) #56% female, 44% male


#Graphic relation about age of consumers
dev.new()
ggplot(customers, aes(Age)) + 
  geom_histogram(aes(y=..density..),
                 fill="white", 
                 color="black",
                 binwidth = 2) +
  scale_x_continuous(breaks = seq(15, 75, 10)) + 
  labs(x = "Age", y = "Density") +
  geom_density(fill="#9C9CEE", 
               alpha=0.5)

mean(customers$Age) # average age= 38.85
nrow(customers[customers$Age < 40,])  # 116 customers are less than 40 years old


#Graphic relation about annual Income
dev.new()
ggplot(customers, aes(AnnualIncome)) + 
  geom_histogram(aes(y=..density..),
                 fill="white", 
                 color="black",
                 binwidth = 6) +
  scale_x_continuous(breaks = seq(0, 140, 20)) + 
  labs(x = "Income in thousands of dollars", y = "Density") +
  ggtitle("Income distribution") +
  geom_density(fill="#9C9CEE", 
               alpha=0.5)
mean(customers$AnnualIncome) #60.56K è la media del reddito annuo percepito
median(customers$AnnualIncome) #61.5 la mediana del reddito annuo percepito


#Graphic relation about spending_score
dev.new()
ggplot(customers, aes(SpendingScore)) + 
  geom_histogram(aes(y=..density..),
                 fill="white", 
                 color="black",
                 binwidth = 5) +
  scale_x_continuous(breaks = seq(0, 101, 10)) + 
  labs(x = "Spending Score", y = "Density") +
  ggtitle("Spending score distribution") +
  geom_density(fill="#9C9CEE", 
               alpha=0.5)

mean(customers$SpendingScore) #50.2 is the average score assigned to customers
median(customers$SpendingScore) #50 is the median

In [ ]:
##### CLUSTER ANALYSIS WITH HIERARCHICAL METHOD

#I select the variables to be considered for 'cluster analysis
my_customers<-customers[,c(2,3,4)] #I decide to choose the variables Age, Annual_Income, and Spending_Score.

my_customers<-as.data.frame(my_customers)

#I calculate the Euclidean distance
d2<-dist(my_customers, method = "euclidean")
summary(d2)

gruppi<-hclust(d2,method="ward.D2")
plot(gruppi)
plot(gruppi$height) #Graph to see the fusioine height of each cluster!

#I build the dendrogram

dev.new()

plot(gruppi,main='Distanza Euclidea-ward.D2',ylab = 'Distanza',
     labels = FALSE, hang = -1,sub = 'ward.D2',frame.plot = TRUE)



#I choose the number of dendrogram groups via the NbClust function

nclust2<-NbClust(my_customers, diss=d2,distance=NULL,method="ward.D2",index="kl")
nclust2$Best.nc #the algorithm suggests 6 classes/clusters
nclust2$Best.partition
rect.hclust(gruppi,k=6,border = 'red')



dev.new()
num_class=6
classi<-cutree(gruppi, k=num_class)  #cutting dendogram into 6 classes
plot(classi)
rect.hclust(gruppi,k=num_class,border = 'red')#drawing red rectangles on clusters
table(classi)

###considering 3 variables the algorithm returns 6 classes having: 1(23), 2(20), 3(32), 4(51), 5(39), 6(35)

#ANALYZE THE DIFFERENT GROUPS: 
gruppo1=my_customers[which(classi==1), ]
gruppo2=my_customers[which(classi==2), ]
gruppo3=my_customers[which(classi==3), ]
gruppo4=my_customers[which(classi==4), ]
gruppo5=my_customers[which(classi==5), ]
gruppo6=my_customers[which(classi==6), ]

#ADD THE GROUP NUMBER TO MY_CUSTOMERS

x1=cbind(classi,my_customers)

#I create a table with the different groups and their averages 

groups=group_by(x1,classi) %>% 
  summarise(
    count = n(),
    mean_Age = mean(Age),
    mean_AnnualIncome=mean(AnnualIncome),
    mean_SpendingScore=mean(SpendingScore),
    na.rm = TRUE)

View(groups)

#I create graphs to report the results of the GROUPS table.

dev.new()
ggboxplot(x1,x="classi",y="Age",color="classi",
          palette=c("#00AFBB", "#E7B800","blue","pink","yellow","lightgreen"),main="group age")
dev.new()
ggboxplot(x1,x="classi",y="AnnualIncome",color="classi",
          palette=c("#00AFBB", "#E7B800","blue","pink","yellow","lightblue"),main="reddito medio annuo")
dev.new()
ggboxplot(x1,x="classi",y="SpendingScore",color="classi",
          palette=c("#00AFBB", "#E7B800","blue","pink","yellow","lightgreen"),main="score group")

##I calculate the silhouette score of the groups
si_6 <- silhouette(cutree(gruppi, k = 6), daisy(my_customers))
dev.new()
plot(si_6, nmax = 80, cex.names = 0.5) ##silhouette score= 0.44   GOOD!


In [ ]:
##### CLUSTER ANALYSIS WITH K-MEANS

#I select the variables to be considered for cluster analysis

k_customers<-customers[,c(2,3,4)] #I consider Age, Annual_Income and SpendingScore.

fit<-kmeans(k_customers,centers = 6,nstart = 10)#I chose 6 groups based on the silhouette score and the pseudo-F
print(fit)

print(fit$cluster[10]) # tells me which group unit 10 is in.

print(fit$cluster) 
print(fit$centers) #coordinates of the centroids of the groups 

k.means<-fit$cluster


##calculate the Pseudo-F index
require(clusterSim)
minC <- 2
maxC <- 10
res <- sapply(minC:maxC,
              function(nc, data) index.G1(data, kmeans(data,centers = nc)$cluster),
              data = k_customers)
ggp <- ggplot(data=data.frame(x=2:(length(res)+1), y= res), mapping = aes(x=x,y=y)) + 
  geom_point() + 
  geom_line() +
  xlab("Numero di cluster") +
  ylab("Statistica pseudo-F di Calinski-Harabasz")
print(ggp)
index.G1(k_customers,fit$cluster) #Optimal PseudoF in 6 clusters: 166.72, THE HIGHEST!




## cluster visualization
dev.new()
clusplot(k_customers,
         k.means,
         lines = 0,
         shade = TRUE,
         color = TRUE,
         labels = 2,
         plotchar = FALSE,
         span = TRUE,
         main = paste('Clusters of customers'),
         xlab = 'Annual Income',
         ylab = 'Spending Score')

library(factoextra)
dev.new()
fviz_cluster(object = fit,
               data = k_customers,
               show.clust.cent = TRUE,
               ellipse.type = "euclid",
               star.plot = TRUE,
               labelsize = 0) +
  scale_color_brewer(palette = "Set2") +
  labs(title = "Clustering K-Means") +
  theme_light() +
  theme(legend.position = "none")

##Silhouette score of groups
pr=kmeans(k_customers,6)
si=silhouette(pr)
dev.new()
plot(si) #Average silhouette score= 0.45  good!!
str(si)

for(k in 2:6){
  dev.new()
  plot(silhouette(pam(k_customers, k=k)), main = paste("k = ",k), do.n.k=FALSE)
  mtext("PAM(K_customers)",
        outer = TRUE, font = par("font.main"), cex = par("cex.main")); 
  #frame()
} #the best silhouette is obtained with 6 groups: 0.45!!! As the groups decrease, the silhouette score decreases


###Alternative algorithm for group formation: PAM()
pr=pam(k_customers,6)
si=silhouette(pr) ##silhouette score 0.45 wirh 6 groups!!
dev.new()
plot(si)
str(si)